In [ ]:
# First: pip install wheel

# Next: Download Numpy and Scipy form Gholke's repo Numpy and SciPy


# Then:

# pip install numpy_package.whl
# pip install scipy_package.whl
# https://towardsdatascience.com/6-tips-to-optimize-an-nlp-topic-model-for-interpretability-20742f3047e2

In [1]:
# Importing modules
import random
import os
from pathlib import Path
from pprint import pprint
import logging
from importlib import reload  # Not needed in Python 2
import tqdm

import numpy as np
import scipy
import pandas as pd
import pickle

import nltk
import spacy
from nltk.corpus import stopwords
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel, LdaModel
from gensim.models.callbacks import PerplexityMetric, ConvergenceMetric, CoherenceMetric

reload(logging)
logging.basicConfig(filename='model_callbacks.log',format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
# logging.basicConfig(filename='gensim.log',
#                     format="%(asctime)s:%(levelname)s:%(message)s",
#                     level=logging.NOTSET)

In [2]:
DATA_DIR = Path('./data')
def load(filename):
    f = open(DATA_DIR/filename,"rb")
    return pickle.load(f)
    
def save(data, filename):
    with open(DATA_DIR/filename, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:

file = open("moties_processed_df.pickle","rb")
df = pickle.load(file)
print('before removal empty texts',len(df))

# remove moties without text
mask = (df['Text']=='') | (df['Text'].isna())
df = df.loc[~mask]
print('after removal empty texts',len(df))

29484

In [5]:
indieners = {indiener[-1].lower() for indiener in df['Indiener_persoon'].str.split() if indiener}
# df = df['Text'].reset_index()
df.reset_index(inplace=True)
df.iloc[2000]['Text']

'2\nTweede Kamer der Staten-Generaal\nVergaderjaar 2009–2010\n31 371 Kredietcrisis\nNr. 293 MOTIE VAN HET LID TONY VAN DIJCK\nVoorgesteld 16 december 2009\nDe Kamer,\ngehoord de beraadslaging,\nbesluit een parlementaire enquête te houden naar het faillissement van de\nDSB Bank waarbij alle betrokkenen onder ede kunnen worden gehoord,\nen gaat over tot de orde van de dag.\nTony van Dijck\nKST138845\n0910tkkst31371-293\nISSN0921-7371\nSduUitgevers\nTweede Kamer, vergaderjaar 2009–2010, 31 371, nr. 293\n’s-Gravenhage2009'

In [6]:
# https://github.com/kapadias/mediumposts/blob/master/natural_language_processing/topic_modeling/notebooks/Evaluate%20Topic%20Models.ipynb
error = []
def remove_indieners(doc):
    try:
        return re.search(pattern, doc).groups()[0]
    except:
        error.append(doc)
        return 'None'

import re
# Remove punctuation
df['Text'] = df['Text'].map(lambda x: re.sub('[,\.!?;]', '', x))

df['Text'] = df['Text'].map(lambda x: re.sub(r'-\n', '', x))
df['Text'] = df['Text'].map(lambda x: re.sub(r'\n', ' ', x))
# Convert the titles to lowercase
df['Text'] = df['Text'].map(lambda x: x.lower())
pattern = re.compile("(?:voorgesteld|gehoord de beraadslaging)(.+) orde van de dag")
# df['Text'] = df['Text'].apply(lambda x: remove_indieners(x))
# df['Text'] = df['Text'].str[64:-123]
# Print out the first rows of papers
df['Text'][0]

'2 tweede kamer der staten-generaal vergaderjaar 2008–2009 23 432 de situatie in het midden-oosten nr 268 motie van het lid pechtold voorgesteld 14 januari 2009 de kamer gehoord de beraadslaging constaterende dat nederland en denemarken binnen de europese unie een voorstel hebben gedaan voor een effectief grenstoezicht op de gazaans-egyptische grens overwegende dat een dergelijk grenstoezicht de invoer van humanitaire hulp en normale goederen mogelijk kan maken en kan voorkomen dat hamas zich via de grens herbewapent verzoekt de regering bij het uitwerken van dit initiatief voorstellen te doen voor een stevige opdracht omvang en mandaat van een dergelijk (militair) grenstoezicht opdat deze daadwerkelijk effectief kan zorg dragen voor humanitaire hulp en ontwapening en gaat over tot de orde van de dag pechtold kst127027 0809tkkst23432-268 issn0921-7371 sduuitgevers tweede kamer vergaderjaar 2008–2009 23 432 nr 268 ’s-gravenhage2009'

In [7]:
len(df.loc[df['Text']=="None"])

0

In [8]:

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = df['Text'].values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:90])

['tweede', 'kamer', 'der', 'staten', 'generaal', 'vergaderjaar', 'de', 'situatie', 'in', 'het', 'midden', 'oosten', 'nr', 'motie', 'van', 'het', 'lid', 'pechtold', 'voorgesteld', 'januari', 'de', 'kamer', 'gehoord', 'de', 'beraadslaging', 'constaterende', 'dat', 'nederland', 'en', 'denemarken', 'binnen', 'de', 'europese', 'unie', 'een', 'voorstel', 'hebben', 'gedaan', 'voor', 'een', 'effectief', 'grenstoezicht', 'op', 'de', 'gazaans', 'egyptische', 'grens', 'overwegende', 'dat', 'een', 'dergelijk', 'grenstoezicht', 'de', 'invoer', 'van', 'humanitaire', 'hulp', 'en', 'normale', 'goederen', 'mogelijk', 'kan', 'maken', 'en', 'kan', 'voorkomen', 'dat', 'hamas', 'zich', 'via', 'de', 'grens', 'herbewapent', 'verzoekt', 'de', 'regering', 'bij', 'het', 'uitwerken', 'van', 'dit', 'initiatief', 'voorstellen', 'te', 'doen', 'voor', 'een', 'stevige', 'opdracht', 'omvang']


In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=20, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], min_count=20, threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [10]:
# NLTK Stop words


nltk.download('stopwords')
stop_words = stopwords.words('dutch')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stop_words.extend(indieners)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jesse\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
!python -m spacy download nl_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('nl_core_news_sm')


In [13]:

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("nl_core_news_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['kamer', 'staat', 'generaal', 'vergaderjaar', 'situatie', 'midden', 'nr', 'motie', 'lid', 'voorstellen', 'januari', 'kamer', 'horen', 'beraadslaging', 'constateren', 'nederland', 'denemarken', 'voorstel', 'doen', 'effectief', 'grenstoezicht', 'gazaan', 'egyptisch', 'grens', 'overwegen', 'grenstoezicht', 'invoer', 'normaal', 'goed', 'mogelijk']


** **
#### Step 4: Data transformation: Corpus and Dictionary
** **

The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

In [14]:


# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)


print('len of dictionary before pruning\t: ', len(id2word))
id2word.filter_extremes(no_below=5, no_above=0.12) #minimally 10 occurrences and not in more than 30% of documents
print('len of dictionary after pruning\t: ',len(id2word))
r = random.choice(id2word.keys())
print('example from id2word dict:\t', r, id2word[r])


# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print('example document translated into bag of words ', corpus[:1][0][:30])

texts = [[id2word[word_id] for word_id, freq in doc] for doc in corpus]

len of dictionary before pruning	:  37446
len of dictionary after pruning	:  11672
example from id2word dict:	 2999 onderschatten
example document translated into bag of words  [(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1)]


In [15]:
# save((id2word, corpus, texts), 'corpus_dict_texts_full_remove_indieners_no_above012')

In [3]:

id2word, corpus, texts = load('corpus_dict_texts_full_remove_indieners_no_above012')

In [5]:
t = df.iloc[20000]['Text']
t

'2\nTweede Kamer der Staten-Generaal\nVergaderjaar 2017–2018 \n31 311 Zelfstandig ondernemerschap \nNr. 201   MOTIE VAN HET LID VAN DER LEE C.S. \nVoorgesteld 14 maart 2018 \nDe Kamer, \ngehoord de beraadslaging, \nconstaterende dat de Nederlandse Vereniging van Banken (NVB) in haar \nGedragscode Kleinzakelijke Financiering beperkt aandacht schenkt aan \nalternatieve vormen van financiering; \noverwegende dat alternatieve vormen van financiering, zoals crowd-\nfunding, steeds vaker worden gebruikt door mkb’ers en het zowel in het \nbelang van ondernemers als investeerders wenselijk is om ook op dit punt \nzelfregulering te bevorderen; \nverzoekt de regering, hierover in overleg te treden met de NVB en de \nalternatieve financiers, bij voorkeur voordat de Gedragscode Kleinzake-\nlijke Financiering definitief in werking treedt, \nen gaat over tot de orde van de dag. \nVan der Lee \nPaternotte \nMoorlag \nWörsdörfer \nAmhaouch\n \n \n \n \nkst-31311-201\nISSN 0921 - 7371\n’s-Gravenhage 20

In [6]:
texts[20000]

['belang',
 'gebruiken',
 'overleg',
 'voorkeur',
 'vorm',
 'wenselijk',
 'steeds',
 'bevorderen',
 'beperken',
 'bank',
 'ondernemer',
 'ers',
 'treden',
 'punt',
 'nvb',
 'definitief',
 'vaker',
 'hierover',
 'schenken',
 'alternatief',
 'financiering',
 'aandacht',
 'investeerder',
 'vereniging',
 'gedragscod',
 'zelfstandig_ondernemerschap',
 'gedragscoen',
 'zelfregulering',
 'financier',
 'werking_treeden',
 'kleinzakelijk',
 'worsdorf']

** **
#### Step 5: Base Model 
** **

We have everything required to train the base LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well. Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior (we'll use default for the base model).

chunksize controls how many documents are processed at a time in the training algorithm. Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory.

passes controls how often we train the model on the entire corpus (set to 10). Another word for passes might be "epochs". iterations is somewhat technical, but essentially it controls how often we repeat a particular loop over each document. It is important to set the number of "passes" and "iterations" high enough.

In [53]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

** **
The above LDA model is built with 10 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

You can see the keywords for each topic and the weightage(importance) of each keyword using `lda_model.print_topics()`

In [54]:

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.065*"xvi" + 0.043*"jaar" + 0.042*"orde" + 0.038*"ministerie" + '
  '0.033*"xvivaststelling" + 0.033*"sport" + 0.033*"volksgezondheid" + '
  '0.028*"risico" + 0.028*"zorg" + 0.022*"mening"'),
 (1,
  '0.070*"ziekenhuis" + 0.058*"constateren" + 0.034*"nederland" + '
  '0.032*"stellen" + 0.030*"bevolking" + 0.030*"algemeen" + 0.030*"vlissing" + '
  '0.026*"hama" + 0.024*"middel" + 0.024*"positionering"'),
 (2,
  '0.113*"israel" + 0.088*"constateren" + 0.056*"situatie" + '
  '0.051*"middenoosten" + 0.049*"hama" + 0.045*"europees" + 0.035*"gaza" + '
  '0.025*"nederland" + 0.024*"relatie" + 0.024*"gebruiken"'),
 (3,
  '0.061*"mening" + 0.038*"toelating" + 0.038*"constateren" + 0.029*"dagvan" + '
  '0.027*"maand" + 0.026*"nederlands" + 0.025*"bestaan" + 0.025*"recht" + '
  '0.024*"kabinet" + 0.023*"maken"'),
 (4,
  '0.050*"humanitair" + 0.048*"situatie" + 0.040*"internationaal" + '
  '0.039*"hulp" + 0.036*"middenoosten" + 0.036*"geweld" + 0.034*"gaza" + '
  '0.029*"mogelijk" + 0.028*

#### Compute Model Perplexity and Coherence Score

Let's calculate the baseline coherence score

** **
#### Step 6: Hyperparameter tuning
** **
First, let's differentiate between model hyperparameters and model parameters :

- `Model hyperparameters` can be thought of as settings for a machine learning algorithm that are tuned by the data scientist before training. Examples would be the number of trees in the random forest, or in our case, number of topics K

- `Model parameters` can be thought of as what the model learns during training, such as the weights for each word in a given topic.

Now that we have the baseline coherence score for the default LDA model, let's perform a series of sensitivity tests to help determine the following model hyperparameters: 
- Number of Topics (K)
- Dirichlet hyperparameter alpha: Document-Topic Density
- Dirichlet hyperparameter beta: Word-Topic Density

We'll perform these tests in sequence, one parameter at a time by keeping others constant and run them over the two difference validation corpus sets. We'll use `C_v` as our choice of metric for performance comparison 

In [6]:
from gensim.models.wrappers import LdaMallet
path_to_mallet_binary = "mallet-2.0.8\\bin\\mallet"

# supporting function
def compute_coherence_values(corpus, id2word, k, a='auto',b='auto',mallet=True):
    if mallet:
        lda_model_mallet = LdaMallet(path_to_mallet_binary, corpus=corpus, num_topics=k, id2word=id2word)
        lda_model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda_model_mallet, gamma_threshold=0.001, iterations=50)
    else:
        lda_model = gensim.models.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=k, 
                                            random_state=100,
                                            chunksize=3000,
                                            passes=7,
                                            alpha=a,
                                            eta=b)
                                            #    eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    
    return (lda_model, coherence_model_lda.get_coherence())

Let's call the function, and iterate it over the range of topics, alpha, and beta parameter values

In [7]:

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 10
max_topics = 30
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = [0.01,0.005,0.001]

# Beta parameter
beta = [0.01, 0.005,0.001]

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
pbar = tqdm.tqdm(total=len(topics_range))

# iterate through number of topics
for k in topics_range:
    for a in ['no']:
        for b in ['no']:
            # get the coherence score for the given parameters
            lda_model, cv_res = compute_coherence_values(corpus, id2word, k, a,b)
            save(lda_model, f'lda_model_{k}_results_{cv_res}.pickle')
            # Save the model results
            model_results['Validation_Set'].append('bla')
            model_results['Topics'].append(k)
            model_results['Alpha'].append(a)
            model_results['Beta'].append(b)
            model_results['Coherence'].append(cv_res)
            print(f'a {a} b {b} k {k} results {cv_res}')
            
            pbar.update(1)
pd.DataFrame(model_results).to_csv('mallet_lda_tuning_results.csv', index=False)
pbar.close()


  5%|▌         | 1/20 [04:00<1:16:00, 240.04s/it]a no b no k 10 results 0.4814481592719376

 10%|█         | 2/20 [08:12<1:13:05, 243.65s/it]a no b no k 11 results 0.46821130856680393

 15%|█▌        | 3/20 [12:03<1:07:59, 239.95s/it]a no b no k 12 results 0.4735753392219608

 20%|██        | 4/20 [15:49<1:02:53, 235.85s/it]a no b no k 13 results 0.4797857694574094

 25%|██▌       | 5/20 [19:17<56:52, 227.51s/it]  a no b no k 14 results 0.49479646170597275

 30%|███       | 6/20 [23:02<52:53, 226.68s/it]a no b no k 15 results 0.49118848890462447

 35%|███▌      | 7/20 [26:39<48:29, 223.82s/it]a no b no k 16 results 0.4864493755703289

 40%|████      | 8/20 [31:09<47:30, 237.52s/it]a no b no k 17 results 0.48496873946309926

 45%|████▌     | 9/20 [34:47<42:28, 231.72s/it]a no b no k 18 results 0.5009906888030362

 50%|█████     | 10/20 [38:10<37:10, 223.10s/it]a no b no k 19 results 0.50679160623646

 55%|█████▌    | 11/20 [42:04<33:58, 226.50s/it]a no b no k 20 results 0.4926948157088

In [76]:
# Set up the callbacks loggers
perplexity_logger = PerplexityMetric(corpus=corpus, logger='shell')
convergence_logger = ConvergenceMetric(logger='shell')
coherence_logger = CoherenceMetric(corpus=corpus, logger='shell', coherence = 'c_v',texts=texts)
coherence_logger_umass = CoherenceMetric(corpus=corpus, logger='shell', coherence = 'u_mass', texts=texts)

In [ ]:
import re
import matplotlib.pyplot as plt
p = re.compile("(-*\d+\.\d+) per-word .* (\d+\.\d+) perplexity")
matches = [p.findall(l) for l in open('gensim.log')]
matches = [m for m in matches if len(m) > 0]
tuples = [t[0] for t in matches]
perplexity = [float(t[1]) for t in tuples]
liklihood = [float(t[0]) for t in tuples]
iter = list(range(0,len(tuples)*10,10))
plt.plot(iter,liklihood,c="black")
plt.ylabel("log liklihood")
plt.xlabel("iteration")
plt.title("Topic Model Convergence")
plt.grid()
plt.savefig("convergence_liklihood.pdf")
plt.close()

In [11]:
# from pprint import pprint
lda_model = load('lda_model_15_results_0.4576345837725311.pickle')
# # Print the Keyword in the 10 topics
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

In [28]:
import pandas as pd
with open('model_callbacks.log') as f:
    lines = [line.split() for line in f.read().splitlines()]
metrics = [[],[],[],[]]

for i in range(len(lines)):
    metrics[i % 4].append(float(lines[i][-1]))
metrics = pd.DataFrame(metrics).T
metrics.columns = ['conv','perp','coh_cv','coh_umass']
metrics.reset_index(inplace=True)
import altair as alt


alt.Chart(...)

In [37]:
alt.Chart(metrics).mark_line().encode(
    x= 'index:O',
    y='coh_cv:Q')

alt.Chart(...)

** **
#### Step 8: Visualize Results
** **

In [17]:
num_topics = 20

lda_model = gensim.models.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=3000,
                                           passes=7)
                                          #  eval_every=5,callbacks=[convergence_logger, perplexity_logger, coherence_logger,coherence_logger_umass])
                                        #    eta=1/num_topics)

In [5]:
lda_model = load('lda_model_19_results_0.4969000869602997.pickle')

In [30]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from IPython.core.display import display, HTML

# Visualize the topics
pyLDAvis.enable_notebook()
num_topics = 28
LDAvis_data_filepath = os.path.join('./results/ldavis_tuned_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, 'results/ldavis_tuned_'+ str(num_topics) +'.html')

C:\ProgramData\Miniconda3\lib\site-packages\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\ProgramData\Miniconda3\lib\site-packages\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


In [31]:
display(HTML("<style>.container { max-width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.output_area { max-width:100% !important; }</style>"))
display(HTML("<style>.input_area { max-width:100% !important; }</style>"))
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
7     -0.010140  0.037701       1        1  5.322205
10    -0.241563 -0.124007       2        1  5.293393
11     0.102002 -0.087787       3        1  5.292212
5     -0.171153  0.111301       4        1  5.283135
18     0.137522  0.071891       5        1  5.280742
8      0.128093  0.134174       6        1  5.274700
17     0.133341  0.021627       7        1  5.269592
9      0.062464 -0.094723       8        1  5.267756
2      0.110063 -0.059347       9        1  5.266109
14     0.110318 -0.049639      10        1  5.264310
3      0.115941  0.065030      11        1  5.260671
12    -0.082435  0.202372      12        1  5.260557
16    -0.113385  0.102236      13        1  5.253698
4      0.050545 -0.190362      14        1  5.248962
15     0.049713 -0.246812      15        1  5.248141
1     -0.201462  0.034371      16        1  5.233729
0     -0.005103  0.165650      17        1  5.232288
13    -0.199468 -0.174457      18        1  5.227884
6      0.024709  0.080782      19        1  5.219916, topic_info=           Term         Freq        Total Category  logprob  loglift
25         zorg  5536.000000  5536.000000  Default  30.0000  30.0000
149    europees  4817.000000  4817.000000  Default  29.0000  29.0000
1519  onderwijs  5299.000000  5299.000000  Default  28.0000  28.0000
433    gemeente  3881.000000  3881.000000  Default  27.0000  27.0000
54    onderzoek  4260.000000  4260.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
337       nemen   596.716605  4014.206279  Topic19  -4.5080   1.0465
116    toekomst   384.082438   944.450942  Topic19  -4.9486   2.0529
2208   concreet   370.673076   891.328279  Topic19  -4.9841   2.0753
75       waarin   377.377757  1328.556556  Topic19  -4.9662   1.6941
1122      basis   375.462134  1346.341108  Topic19  -4.9713   1.6757

[1585 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1417      3  1.001796  aanbesteding
3627     19  1.000545   aanbeveling
2112      5  0.006791        aanbod
2112      6  0.278430        aanbod
2112     11  0.715316        aanbod
...     ...       ...           ...
594      11  0.028299        zullen
594      19  0.000629        zullen
5354      4  1.007576           zus
6491     17  0.995906  zwangerschap
4162      1  1.001000        zzp_er

[2337 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 11, 12, 6, 19, 9, 18, 10, 3, 15, 4, 13, 17, 5, 16, 2, 1, 14, 7])

In [ ]:
def word_frequency(word):
    return sum([1 for t in texts if word in t])/len(texts)
word = 'financieel'
word_frequency(word)

In [44]:
def extract_topics(lda_model, corpus):
    for doc in corpus[0:3]:
        print(sorted(lda_model.get_document_topics(doc), key=lambda x: -x[1])) 
    
    probs = [sorted(lda_model.get_document_topics(doc), key=lambda x: -x[1]) for doc in corpus]
    highest_prob = [doc[0][1] for doc in probs]
    topics = [str(doc[0][0]) for doc in probs]
    print(pd.DataFrame(topics).value_counts())
    print(pd.DataFrame(highest_prob).describe())
    return topics
topics = extract_topics(lda2, corpus)

[(13, 0.15528459820517304), (5, 0.08225840199416362), (6, 0.0753635934507883), (4, 0.06145462311686514), (10, 0.05522516872402901), (0, 0.053704022913041376), (1, 0.05264318466434439), (11, 0.04679254548355448), (16, 0.046113554462242065), (9, 0.0411847472611575), (14, 0.03975432046939636), (7, 0.038868641704887984), (2, 0.038458568816972676), (18, 0.038212709239669435), (17, 0.035979417226431346), (3, 0.03553950841073199), (12, 0.035364579808803004), (8, 0.03448668813167783), (15, 0.03331112591607039)]
[(13, 0.2497302878773742), (12, 0.12484864306295111), (5, 0.07332933255803031), (6, 0.07042026222278479), (16, 0.05106820515437053), (1, 0.05028607827219443), (11, 0.042578873636171695), (18, 0.03711988381844349), (2, 0.033715172137801), (17, 0.030379296988030616), (8, 0.03014332019919721), (9, 0.029229424619212716), (4, 0.028046367555082476), (0, 0.027382695457408256), (10, 0.026974555708107844), (14, 0.026061441501876257), (3, 0.023662014002135984), (7, 0.02337199578368185), (15, 0.02

In [35]:
df['Topic'] = topics
assert len(topics) == len(df)

NameError: name 'df' is not defined

In [14]:
save(df[:1000], 'df_including_topics.pickle')

In [26]:
from gensim.test.utils import common_corpus, common_dictionary
# vector = model[common_corpus[0]]  # LDA topics of a documents

In [28]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    
coherence_model_lda.get_coherence()

0.5051945749902539

In [24]:
model.alpha

array([0.00052632, 0.00052632, 0.00052632, 0.00052632, 0.00052632,
       0.00052632, 0.00052632, 0.00052632, 0.00052632, 0.00052632,
       0.00052632, 0.00052632, 0.00052632, 0.00052632, 0.00052632,
       0.00052632, 0.00052632, 0.00052632, 0.00052632])

In [29]:
lda2 = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(model, gamma_threshold=0.001, iterations=50)
coherence_model_lda = CoherenceModel(model=lda2, texts=texts, dictionary=id2word, coherence='c_v')
    
coherence_model_lda.get_coherence()

0.5051945749902539